In [ ]:
!pip install requests pandas numpy matplotlib


In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime


In [ ]:
API_KEY = "e8b43912-6603-413c-b544-3ca7f47cd06b"
API_DOMAIN = "https://api.elections.kalshi.com"  # ✅ Use this domain for most backtesting

def get_headers():
    return {
        "Authorization": f"Bearer {API_KEY}",
        "Accept": "application/json"
    }


In [ ]:
# def get_resolved_main_exchange():
#     url = "https://api.elections.kalshi.com/trade-api/v2/markets/"
#     headers = {
#         "Authorization": f"Bearer {API_KEY}",
#         "Accept": "application/json"
#     }
#     params = {"status": "resolved"}

#     response = requests.get(url, headers=headers, params=params)

#     if response.status_code != 200:
#         print(f"❌ Failed to fetch markets: {response.status_code}")
#         print(response.text)
#         return []

#     resolved = response.json().get("markets", [])
#     print(f"✅ Found {len(resolved)} resolved markets (main exchange).")
#     return resolved

In [ ]:
# markets = get_resolved_main_exchange()
# df = pd.DataFrame(markets)
# df = df.dropna(subset=["settlement_price", "last_price_dollars"])
# df = df[df["market_type"] == "binary"]
# display(df[["ticker", "title", "last_price_dollars", "settlement_price"]].head())

In [ ]:
def get_resolved_markets(api_domain="https://api.elections.kalshi.com"):
    url = f"{api_domain}/trade-api/v2/markets/"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Accept": "application/json"
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"❌ Failed to fetch markets: {response.status_code}")
        print(response.text)
        return []

    all_markets = response.json().get("markets", [])

    # ✅ Now manually filter: resolved + has settlement_price
    resolved = [
        m for m in all_markets
        if m.get("status") == "resolved"
        and m.get("settlement_price") is not None
        and m.get("market_type") == "binary"
    ]

    print(f"✅ Found {len(resolved)} resolved binary markets.")
    return resolved


In [ ]:
markets = get_resolved_markets()
if not markets:
    print("⚠️ No resolved markets found.")
else:
    df = pd.DataFrame(markets)
    display(df[["ticker", "title", "last_price_dollars", "settlement_price"]].head())


In [ ]:
expected_cols = ["ticker", "title", "close_time", "expiration_time", "last_price_dollars", "settlement_price"]
missing_cols = [col for col in expected_cols if col not in df.columns]
if missing_cols:
    print(f"⚠️ Missing columns in API response: {missing_cols}")
else:
    df["close_time"] = pd.to_datetime(df["close_time"])
    df["expiration_time"] = pd.to_datetime(df["expiration_time"])
    df["last_price_dollars"] = df["last_price_dollars"].astype(float)
    df["settlement_price"] = df["settlement_price"].astype(float)
    display(df.head())


In [ ]:
#Enter Strategy:
def simple_strategy(row):
    # Buy YES if last_price < 0.30
    if row["last_price_dollars"] < 0.30:
        entry_price = row["last_price_dollars"]
        outcome = row["settlement_price"]
        pnl = outcome - entry_price  # 1 - entry if YES wins, -entry if NO wins
        return pnl
    else:
        return 0  # no trade


In [ ]:
df["pnl"] = df.apply(simple_strategy, axis=1)

# Filter to show only trades that were taken
trades = df[df["pnl"] != 0]

# Basic performance metrics
total_trades = len(trades)
win_trades = trades[trades["pnl"] > 0]
loss_trades = trades[trades["pnl"] < 0]

win_rate = len(win_trades) / total_trades if total_trades else 0
total_pnl = trades["pnl"].sum()
avg_pnl = trades["pnl"].mean()
profit_factor = win_trades["pnl"].sum() / abs(loss_trades["pnl"].sum()) if not loss_trades.empty else np.inf

print(f"📈 Strategy Backtest Results:")
print(f"Total trades: {total_trades}")
print(f"Win rate: {win_rate:.2%}")
print(f"Total PnL: ${total_pnl:.2f}")
print(f"Average PnL per trade: ${avg_pnl:.2f}")
print(f"Profit Factor: {profit_factor:.2f}")


In [ ]:
trades["cum_pnl"] = trades["pnl"].cumsum()

plt.figure(figsize=(12, 6))
plt.plot(trades["expiration_time"], trades["cum_pnl"], marker="o")
plt.title("Cumulative PnL Over Time")
plt.xlabel("Expiration Time")
plt.ylabel("Cumulative PnL ($)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df.to_csv("kalshi_historical_backtest.csv", index=False)
